### Data Cleaning and Preprocessing

In [1]:
# General imports
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
# Load data
LC_data = pd.read_csv(r'/Users/regina/Documents/UWO/Banking Analytics/LCFinal.csv', 
                      low_memory = False)
LC_data.shape

(2260701, 148)

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Sampling - 50%
LC_data_smp = LC_data.sample(frac = 0.5, random_state = 1)
LC_data_smp.shape

(1130350, 148)

In [4]:
# Find how many missing values in each variable (in each column)
# Get the variable names if the variable has more than 25% missing values
VblDrop = LC_data_smp.isnull().sum().index[LC_data_smp.isnull().sum()/len(LC_data_smp) > 0.25].tolist()

LC_data_keep = LC_data_smp.drop(VblDrop, axis = 1)

In [5]:
# Find how many missing values in each observation (in each row)
# Drop the obervations that have missing values more than 25% (in each row)
LC_data_keep.dropna(thresh=0.75*LC_data_keep.shape[1], inplace=True)

In [6]:
LC_data_keep.shape

(1095184, 90)

In [7]:
# Detect and remove outliers
Q1 = LC_data_keep.quantile(0.25)
Q3 = LC_data_keep.quantile(0.75)
IQR = Q3 - Q1
LC_data_keep_out = LC_data_keep[~((LC_data_keep < (Q1 - 1.5 * IQR))|
                                  (LC_data_keep > (Q3 + 1.5 * IQR))).any(axis=1)]

In [8]:
LC_data_keep_out.shape

(160333, 90)

In [9]:
# Correlation
c = LC_data_keep_out.corr().abs()

# Select upper triangle of correlation matrix
upper = c.where(np.triu(np.ones(c.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.7
to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]
to_drop.remove('total_rec_prncp')

LC_final = LC_data_keep_out.drop(LC_data_keep_out[to_drop], axis=1)

In [10]:
# Drop the variables that do not provide predictive power
to_drop2 = ['Unnamed: 0',
            'id',
            'emp_title',
            'url',
            'title',
            'purpose',
            'zip_code',
            'addr_state',
            'earliest_cr_line',
            'initial_list_status',
            'last_pymnt_d',
            'last_credit_pull_d',
            'disbursement_method']

LC_use = LC_final.drop(LC_final[to_drop2], axis=1)

In [11]:
# Fill missing value with median value
LC_use.fillna(LC_use.median(), inplace = True)

In [12]:
# Dummy Coding 1-default;0-good
# 'term'
def term(x):
    if not(pd.isna(x)):
        return int(x[1:3])
    
LC_use['term'] = LC_use.apply(lambda x: 
                              term(x['term']), axis = 1)

In [13]:
# 'pymnt_plan'
pd.unique(LC_use['pymnt_plan'].values.ravel()).tolist()

['n', 'y']

In [14]:
def pp(x):
    if not(pd.isna(x)):
        if x=='n':
            a=1
        else: a=0
    return a

LC_use['pymnt_plan'] = LC_use.apply(lambda x:
                                    pp(x['pymnt_plan']), axis = 1)

In [15]:
# 'emp_length'
pd.unique(LC_use['emp_length'].values.ravel()).tolist()

['8 years',
 '3 years',
 nan,
 '10+ years',
 '< 1 year',
 '5 years',
 '9 years',
 '1 year',
 '4 years',
 '2 years',
 '6 years',
 '7 years']

In [16]:
def emp(x):
    if not(pd.isna(x)):
        if x=='< 1 year':
            a=10
        elif x=='1 year':
            a=9
        elif x=='2 years':
            a=8
        elif x=='3 years':
            a=7
        elif x=='4 years':
            a=6
        elif x=='5 years':
            a=5
        elif x=='6 years':
            a=4
        elif x=='7 years':
            a=3
        elif x=='8 years':
            a=2
        elif x=='9 years':
            a=1    
        else: a = 0
        return a

LC_use['emp_length'] = LC_use.apply(lambda x:
                                    emp(x['emp_length']), axis = 1)
LC_use['emp_length'].fillna(LC_use['emp_length'].mode()[0], inplace = True)

In [17]:
# 'home_ownership'
pd.unique(LC_use['home_ownership'].values.ravel()).tolist()

['RENT', 'MORTGAGE', 'OWN', 'ANY', 'OTHER', 'NONE']

In [18]:
def homeown(x):
    if not(pd.isna(x)):
        if x=='RENT':
            a=3
        elif x=='MORTGAGE':
            a=2
        elif x=='OWN':
            a=1
        else:
            a=float('nan')
        return a

LC_use['home_ownership'] = LC_use.apply(lambda x:
                                        homeown(x['home_ownership']), axis = 1)
LC_use['home_ownership'].fillna(LC_use['home_ownership'].mode()[0], inplace = True)

In [19]:
# 'loan_status'
LC_use = LC_use[(LC_use.issue_d!='Dec-2018')|
                (LC_use.issue_d!='Nov-2018')|
                (LC_use.issue_d!='Oct-2018')|
                (LC_use.issue_d!='Sep-2018')|
                (LC_use.issue_d!='Aug-2018')|
                (LC_use.issue_d!='Jul-2018')|
                (LC_use.issue_d!='Jun-2018')|
                (LC_use.issue_d!='May-2018')|
                (LC_use.issue_d!='Apr-2018')|
                (LC_use.issue_d!='Mar-2018')|
                (LC_use.issue_d!='Feb-2018')|
                (LC_use.issue_d!='Jan-2018')]
pd.unique(LC_use['loan_status'].values.ravel()).tolist()

['Current',
 'Fully Paid',
 'Charged Off',
 'Late (16-30 days)',
 'Late (31-120 days)',
 'In Grace Period',
 'Default']

In [20]:
def loan_status(x):
    if not(pd.isna(x)):
        if (x=='Charged Off')|(x=='Default'):
            a=1
        else: a=0
    return a

LC_use['Default'] = LC_use.apply(lambda x:
                                 loan_status(x['loan_status']), axis = 1)

In [21]:
# 'application_type'
pd.unique(LC_use['application_type'].values.ravel()).tolist()

['Individual', 'Joint App']

In [22]:
def app(x):
    if not(pd.isna(x)):
        if x=='Individual':
            a=2
        else: a=1
    return a

LC_use['application_type'] = LC_use.apply(lambda x:
                                          app(x['application_type']),axis = 1)

In [23]:
# 'hardship_flag'
pd.unique(LC_use['hardship_flag'].values.ravel()).tolist()

['N', 'Y']

In [24]:
def hardship(x):
    if not(pd.isna(x)):
        if x=='Y':
            a=1
        else: a=0
    return a

LC_use['hardship_flag'] = LC_use.apply(lambda x:
                                       hardship(x['hardship_flag']),axis = 1)

In [25]:
# 'debt_settlement_flag'
pd.unique(LC_use['debt_settlement_flag'].values.ravel()).tolist()

['N', 'Y']

In [26]:
def debtset(x):
    if not(pd.isna(x)):
        if x=='N':
            a=1
        else: a=0
    return a

LC_use['debt_settlement_flag'] = LC_use.apply(lambda x:
                                              debtset(x['debt_settlement_flag']),axis = 1)

In [27]:
LC_use.head()

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,fico_range_low,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,total_rec_prncp,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,acc_open_past_24mths,bc_open_to_buy,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_bc_tl,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,hardship_flag,debt_settlement_flag,Default
2253821,1500.0,36,13.49,2.0,3.0,50000.0,Oct-2016,Current,1,21.89,0.0,685.0,0.0,8.0,0.0,6871.0,67.4,10.0,340.63,1159.37,0.0,0.0,0.0,50.90,724.0,0.0,2,0.0,0.0,198115.0,3.0,829.0,0.0,0.0,105.0,49.0,3.0,3.0,1.0,27.0,23.0,0.0,3.0,3.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,37629.0,0,1,0
1946526,6500.0,36,9.49,7.0,3.0,50000.0,Sep-2016,Current,1,18.29,0.0,705.0,1.0,8.0,0.0,6106.0,34.9,9.0,1215.04,5284.96,0.0,0.0,0.0,208.19,679.0,0.0,2,0.0,0.0,52906.0,6.0,11394.0,0.0,0.0,39.0,47.0,12.0,10.0,0.0,12.0,6.0,0.0,3.0,3.0,0.0,0.0,0.0,3.0,100.0,0.0,0.0,52906.0,0,1,0
764988,4800.0,36,13.56,0.0,2.0,34500.0,Sep-2018,Current,1,22.33,0.0,685.0,0.0,11.0,0.0,19858.0,79.8,19.0,4128.53,671.47,0.0,0.0,0.0,163.03,664.0,0.0,1,0.0,0.0,127581.0,1.0,3237.0,0.0,0.0,145.0,42.0,32.0,18.0,1.0,32.0,19.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,19858.0,0,1,0
1611065,10000.0,36,20.39,0.0,3.0,119535.0,Apr-2018,Current,1,17.66,0.0,670.0,0.0,9.0,0.0,36645.0,94.4,13.0,7558.73,2441.27,0.0,0.0,0.0,373.63,594.0,0.0,2,0.0,0.0,327519.0,3.0,2155.0,0.0,0.0,171.0,99.0,1.0,1.0,2.0,1.0,21.0,0.0,7.0,7.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,47650.0,0,1,0
1479045,6350.0,36,23.40,2.0,2.0,70000.0,Oct-2018,Current,1,25.72,0.0,705.0,0.0,12.0,0.0,29375.0,74.0,24.0,5708.91,641.09,0.0,0.0,0.0,247.14,699.0,0.0,1,0.0,0.0,457122.0,4.0,2043.0,0.0,0.0,160.0,359.0,2.0,2.0,7.0,2.0,22.0,0.0,5.0,7.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,42092.0,0,1,0


In [28]:
LC_use = LC_use.drop(['issue_d', 'loan_status'], axis = 1)

### LGD model

In [29]:
LC = LC_use

In [30]:
def wLGD(r,a,d,p):
    if not(pd.isna(d)):
        if d==1 and a-p != 0:
            LGD = 1 - (r/(a-p))
        else:
            LGD = 0
        return(LGD)

LC['wLGD'] = LC.apply(lambda x:
                     wLGD(x['recoveries'], x['loan_amnt'],
                     x['Default'], x['total_rec_prncp']), axis=1)

In [31]:
c2 = LC.corr().abs()
upper2 = c2.where(np.triu(np.ones(c2.shape), k=1).astype(np.bool))
to_drop3 = [column for column in upper2.columns if any(upper2[column] > 0.7)]
to_drop3.remove('wLGD')
LC = LC.drop(LC[to_drop3], axis = 1)
LC = LC.drop(['recoveries','loan_amnt','total_rec_prncp', 
              'collection_recovery_fee'], axis = 1)

In [33]:
# Split default and non-default data
# Split train and test data - 70%
Def_data = LC.loc[(LC['Default']==1)]
NonDef_data = LC.loc[(LC['Default']==0)]

Def_data = Def_data.drop(['Default'], axis = 1)
NonDef_data = NonDef_data.drop(['Default'], axis = 1)

import scorecardpy as sc
train_wLGD, test_wLGD = sc.split_df(Def_data.iloc[:,1:],
                                   y = 'wLGD',
                                   ratio = 0.7,
                                   seed = 251053881).values()

In [34]:
# Random Forests
from sklearn.ensemble import RandomForestRegressor
bankloan_rf = RandomForestRegressor(n_estimators=1000, # Number of trees to train
                       criterion='mse', 
                       max_depth=None, 
                       min_samples_split=2, 
                       min_samples_leaf=0.001, # 0.1%
                       min_weight_fraction_leaf=0.0, 
                       max_features='auto', 
                       max_leaf_nodes=None, 
                       min_impurity_decrease=0.0001, # 10^(-3)
                       bootstrap=True, 
                       oob_score=True,  
                       n_jobs=-1, 
                       random_state=251053881, # Seed
                       verbose=1, # 0 for silent training.
                       warm_start=False)

In [35]:
bankloan_rf.fit(train_wLGD.iloc[:,:-1],
                train_wLGD['wLGD'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    1.1s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0001, min_impurity_split=None,
                      min_samples_leaf=0.001, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=-1, oob_score=True, random_state=251053881,
                      verbose=1, warm_start=False)

In [36]:
bankloan_rf.score(train_wLGD.iloc[:,:-1],
                  train_wLGD['wLGD'])

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    0.2s finished


0.1437332408390477

In [37]:
rf_pred_LGD = bankloan_rf.predict(test_wLGD.iloc[:,:-1])

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    0.1s finished


In [40]:
# Calculate MSE
from sklearn.metrics import mean_squared_error
mean_squared_error(test_wLGD.iloc[:,-1], rf_pred_LGD)

1.5075916150361904e-05

In [41]:
# XGBoosting
from sklearn.ensemble import GradientBoostingRegressor 
bankloan_xg = GradientBoostingRegressor(loss='ls', 
                                        learning_rate=0.1, 
                                        subsample=1, 
                                        min_samples_split=2,
                                        min_samples_leaf=1, 
                                        min_weight_fraction_leaf=0.0, 
                                        max_depth=3, 
                                        init=None, 
                                        verbose=1, 
                                        validation_fraction=0.3, 
                                        tol=0.0001 )

In [43]:
bankloan_xg.score(train_wLGD.iloc[:,:-1], 
                  train_wLGD['wLGD'])

0.9996670103634684

In [44]:
# MSE
xg_pred_LGD = bankloan_xg.predict(test_wLGD.iloc[:,:-1])
mean_squared_error(test_wLGD.iloc[:,-1], rf_pred_LGD)

1.5075916150361904e-05